In [ ]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [3]:
import zipfile
import os

# Specify the path to your zip file (assuming it's in the same directory as your script)
zip_file = '/content/homer_bart.zip'

# Get the current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Open and extract the zip file
with zipfile.ZipFile(os.path.join(current_dir, zip_file), 'r') as zip_ref:
    zip_ref.extractall(current_dir)

print(f"Zip file '{zip_file}' extracted successfully to '{current_dir}'.")

Current directory: /content
Zip file '/content/homer_bart.zip' extracted successfully to '/content'.


In [104]:
import os
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from PIL import Image

# Set random seed for reproducibility
SEED = 4247845
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Example function to load BMP images and preprocess
def load_images(folder_path, image_size=(64, 64)):
    images = []
    labels = []
    label_map = {'homer': 0, 'bart': 1}  # Assuming 'homer' folder is class 0, 'bart' folder is class 1

    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            if filename.endswith(".bmp"):
                filepath = os.path.join(root, filename)
                label = os.path.basename(root).lower()
                if label in label_map:
                    img = Image.open(filepath)
                    img = img.resize(image_size)
                    img_array = np.array(img) / 255.0  # Normalize pixel values
                    images.append(img_array)
                    labels.append(label_map[label])

    return np.array(images), np.array(labels)

# Path to your main folder containing 'homer' and 'bart' folders
data_folder = '/content'

# Load images and labels
images, labels = load_images(data_folder)

# Splitting into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=SEED)

# Define the model
model = Sequential([
    Flatten(input_shape=(64, 64, 3)),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(132, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001))
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Implementing early stopping and learning rate reduction
early_stop = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=10, batch_size=32, callbacks=[early_stop, reduce_lr])

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10
8/8 [==============================] - 1s 7ms/step - loss: 1.5263 - accuracy: 0.5537 - lr: 0.0010
Epoch 2/10
8/8 [==============================] - 0s 4ms/step - loss: 1.1487 - accuracy: 0.4835 - lr: 0.0010
Epoch 3/10
8/8 [==============================] - 0s 4ms/step - loss: 0.9986 - accuracy: 0.5785 - lr: 0.0010
Epoch 4/10
8/8 [==============================] - 0s 5ms/step - loss: 0.8582 - accuracy: 0.6322 - lr: 0.0010
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 0.7165 - accuracy: 0.7231 - lr: 0.0010
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 0.6540 - accuracy: 0.7603 - lr: 0.0010
Epoch 7/10
8/8 [==============================] - 0s 5ms/step - loss: 0.7027 - accuracy: 0.7190 - lr: 0.0010
Epoch 8/10
8/8 [==============================] - 0s 4ms/step - loss: 0.7389 - accuracy: 0.7107 - lr: 0.0010
Epoch 9/10
8/8 [==============================] - 0s 6ms/step - loss: 0.5746 - accuracy: 0.7975 - lr: 0.0010
Epoch 10/10
1/1 [==